터미널 창에서 "python -m sglang.launch_server --quantization gptq --model-path /workspace/test/Qwen2-72B-Instruct-GPTQ-Int4 --port 30000 --tp 4 --enable-p2p-check --disable-flashinfer --disable-flashinfer-sampling" 코드를 실행 시킨 후 아래 코드 블럭들을 순차적으로 실행시킨다.

In [4]:
import os
from tqdm import tqdm

import pandas as pd 
import torch
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

import argparse 
from transformers import set_seed
from datasets import Dataset, DatasetDict

import ast
import re

/root/anaconda3/envs/exam/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def make_input(q, c):
    question = []
    template = """질문 : \"{input}\"
                선택지 : 
                A : {choice0}
                B : {choice1}
                C : {choice2}
                D : {choice3}
                """
    for i in range(len(q)):
        conversation = template.replace('{input}', q[i]).replace('{choice0}', c[i][0]).replace('{choice1}', c[i][1]).replace('{choice2}', c[i][2]).replace('{choice3}', c[i][3])
        question.append(conversation)
    return question

def generate_dict(df):
    prompt_q = ""
    prompt_ca = ""
    prompt_ex = ""
    
    instruction_list = [[prompt_q + ' ' + prompt_ca + ' ' + prompt_ex] for _ in range(len(df))]
    question_list = df['문제']
    choices_list = df['선택지']
    dataset_dict = {'instruction': instruction_list, 'question': question_list, 'choices': choices_list}
    dataset = Dataset.from_dict(dataset_dict)
    
    return dataset

In [7]:
options = []
df = pd.read_csv("data/test_data.csv")
text_tm = generate_dict(df)
for i in range(len(text_tm['choices'])):
    options.append(ast.literal_eval(text_tm['choices'][i]))
    
for i in options:
    if len(i) != 4:
        print(i)
        i.append("")

In [4]:
c = []
change_choices = {"A":0,"B":1,"C":2,"D":3}

In [ ]:
import time
import requests
import concurrent.futures

url = "http://127.0.0.1:30000/chat/completions"
headers = {
    "Content-Type": "application/json"
}
model = "/workspace/test/Qwen2-72B-Instruct-GPTQ-Int4"

system_message = {"role": "system", "content": """You are a highly knowledgeable AI assistant with expertise in all fields, and you should provide responses to requests in the most detailed and friendly manner possible. You need to translate questions coming in Korean into English, think through the problem step-by-step, and provide answers accordingly. Analyze the information provided by users carefully to quickly understand their intentions and generate responses that are ethically appropriate, following norms to protect children.
Please provide answers with a higher moral standard than the average American.
Start with "Let's think step by step." and end the sentence with "Answer: ()" only. The correct letter should be included in the parentheses. Carefully examine the given question and options, and if necessary, refer to relevant Wikipedia entries in that field. Provide a detailed, logical explanation of the solution, and make sure to choose one letter among options A, B, C, or D. Even if none of the answers seem correct, you must select one of the letters A, B, C, or D.
"""}

questions = make_input(text_tm["question"], options)

n = 200

result = [questions[i * n:(i + 1) * n] for i in range((len(questions) + n - 1) // n )] 

def create_payload(question):
    return {
        "model": model,
        "sampling_params": {
            "temperature": 0.7,
            "min_p":0.6,
        },
        "messages": [
            system_message,
            {
        'role': 'user',
        'content':"""질문 : 수정 전 잔액시산표의 차변 합계액은 ￦1,000,000이다. 보험료 미경과액 ￦30,000과 이자수익 미수액 ￦20,000을 계상한 후의 수정 후 잔액시산표 차변 합계액은 얼마인가?
        선택지 : 
        A : ￦970,000
        B : ￦990,000
        C : ￦1,020,000
        D : ￦1,050,000
        """
    },
    {
        'role': 'assistant',
        'content': """Question: The debit total of the unadjusted trial balance is ￦1,000,000. What will be the debit total of the adjusted trial balance after accounting for the unearned insurance premium of ￦30,000 and the accrued interest income of ￦20,000?

        Options:
        A : ￦970,000
        B : ￦990,000
        C : ￦1,020,000
        D : ￦1,050,000

        Let's think step by step. We refer to Wikipedia articles on accounting knowledge for help. The given problem requires distinguishing between factors that affect the total of the adjusted trial balance and those that do not. In this case, an unearned insurance premium and accrued interest income have been added. The unearned insurance premium decreases the insurance expense (debit) while increasing the prepaid insurance (asset) by the same amount on the debit side, so it has no effect. Next, the accrued interest income of ￦20,000 represents interest that has not yet been received. This accrued interest is recorded as a debit in the trial balance. Adding the accrued interest income of ￦20,000 to the original debit total of ￦1,000,000 results in an adjusted debit total of ￦1,020,000. Therefore, the correct answer is (C).

Answer: (C)"""
        
    },
    {
        'role': 'user',
        'content':"""질문 : 이차방정식 x2 - 5x + 3 = 0의 두 근이 log5α와 log5β일 때, logαβ + logβα의 값은?
        선택지 : 
        A : 17/3
        B : 19/3
        C : 21/5
        D : 23/5
        """
    },
    {
        'role': 'assistant',
        'content': """Question: If the roots of the quadratic equation x2 - 5x + 3 = 0 are log5α and log5β, what is the value of logαβ + logβα?

        Options:
        A : 17/3
        B : 19/3
        C : 21/5
        D : 23/5

Let's think step by step. We refer to Wikipedia articles on mathematical knowledge for help. The problem is to find the value of \( \log \alpha \beta + \log \beta \alpha \) given that the roots of the quadratic equation \( x^2 - 5x + 3 = 0 \) are \( \log_5 \alpha \) and \( \log_5 \beta \). According to Vieta's formulas for the roots of a quadratic equation, the sum of the roots is \(-\frac{b}{a}\) and the product of the roots is \(\frac{c}{a}\). Here, \(a\), \(b\), and \(c\) are the coefficients of the quadratic equation. For the given equation \( x^2 - 5x + 3 = 0 \), we have \(a = 1\), \(b = -5\), and \(c = 3\), so the sum of the roots is \(5\) and the product of the roots is \(3\). Since \( \log_5 \alpha \) and \( \log_5 \beta \) are the roots, the following conditions hold: 1. \( \log_5 \alpha + \log_5 \beta = 5 \) 2. \( \log_5 \alpha \cdot \log_5 \beta = 3 \). By the properties of logarithms, \( \log \alpha \beta + \log \beta \alpha \) can be expressed as \( \log \alpha^2 \beta^2 \), which is \( \log (\alpha \beta)^2 \). Since \( \alpha \beta = 5^3 \) (from the product of the roots), this is \( \log 5^6 \), which equals \( 6 \cdot \log 5 \). Since \( \log 5 \) is 1, this simplifies to 6. However, considering additional operations and quadratic expansions involving \( \log_5 \alpha \) and \( \log_5 \beta \), the final simplified result in fractional form is \( \frac{19}{3} \). Therefore, the answer is (B).

Answer: (B)"""
        
    },
    {
        'role': 'user',
        'content':"""질문 : (가)에 대한 설명으로 옳지 않은 것은? (가)은/는 의병계열과 애국계몽 운동 계열의 비밀결사가 모여 결성된 조직으로,
      총사령 박상진을 중심으로 독립군 양성을 목적으로 하였다.
        선택지 : 
        A : 북간도에서 무장 독립 단체인 중광단을 조직하였다.
        B : 군자금을 모집하고 친일파를 공격하였다.
        C : 공화제 국가 수립을 지향하였다.
        D : 경상도 일대에서 결성되어 전국 조직으로 확대하였다.
        """
    },
    {
        'role': 'assistant',
        'content': """Question: Which of the following statements about (가) is NOT correct? (가) is an organization formed by a secret alliance of the righteous army faction and the patriotic enlightenment movement faction, with General Park Sang-jin at its center, aimed at training independence fighters.

        Options:
        A : It organized the armed independence group Junggwangdan in North Manchuria.
        B : It raised military funds and attacked pro-Japanese figures.
        C : It aimed to establish a republic.
        D : It was formed in the Gyeongsang-do region and expanded into a nationwide organization.

Let's think step by step. We refer to Wikipedia articles on Korean history for help. the Daehan Gwangbokhoe (대한광복회) was an independence movement organization formed on July 15, 1915, in Dalseong Park, Daegu, by the unification of the righteous army faction and the enlightenment movement faction. This organization aimed to restore national sovereignty and achieve independence by establishing a military academy in Manchuria to train independence fighters and prepare for war against Japan when military strength was sufficient. It also planned to relocate Koreans to Manchuria, develop farmland, and supply food and troops. The Daehan Gwangbokhoe sought to establish a republic, reflecting the widespread republican ideals among independence activists following the advocacy of republicanism by the Sinminhoe. The organization raised military funds and attacked pro-Japanese figures, using commercial organizations disguised as contact points across the country and setting up lodgings and general stores in Dandong and Changchun as contact bases. However, the Daehan Gwangbokhoe did not organize the armed independence group Junggwangdan in North Manchuria. Junggwangdan was an armed struggle organization formed by Daechongyo (Dangun) groups led by Na Cheol and O Gi-ho, and it was the predecessor of the Bukro Gunjeongseo established after the March 1st Movement. The Bukro Gunjeongseo is famous for the unit led by General Kim Jwa-jin in the Battle of Cheongsanri. Therefore, the incorrect statement about (A) is (가). The Daehan Gwangbokhoe did not organize Junggwangdan in North Manchuria, as it was a separate organization. Thus, the answer is (A).

Answer: (A)"""
    },
    {
        'role': 'user',
        'content':"""질문 : (주)한공은 (주)서울 소유의 건물을 3년간 사용하기로 하는 전세계약을 체결하고, 전세금 10,000,000원을 지급하였다.
      (주)한공은 전세금 10,000,000원을 비유동자산 중 어떤 계정과목으로 표시하여야 하는가?
        선택지 : 
        A : 장기외상매출금
        B : 구축물
        C : 임대보증금
        D : 임차보증금
        """
    },
    {
        'role': 'assistant',
        'content': """Question: (주)한공 entered into a three-year lease agreement to use a building owned by (주)서울 and paid a deposit of 10,000,000 won. Under which non-current asset account should (주)한공 classify the deposit of 10,000,000 won?

        Options:
        A : Long-term receivables
        B : Buildings
        C : Security deposit
        D : Lease deposit

Let's think step by step. We refer to Wikipedia articles on economics knowledge for help. a lease deposit refers to the amount paid by the tenant to the landlord to obtain the right to use a building or other property for a specified period. In this case, (주)한공 is the tenant, and (주)서울 is the landlord. The deposit of 10,000,000 won paid by (주)한공 to (주)서울 should be classified as a non-current asset.

Looking at the options:

(A) Long-term receivables refer to receivables that are expected to be collected over the long term, which is not relevant here.
(B) Buildings pertain to properties owned by the company, which is not applicable to the lease deposit.
(C) Security deposit refers to money paid by the landlord to the tenant, which does not match the situation where (주)한공 is paying the deposit.
(D) Lease deposit is the amount paid by the tenant to the landlord for the right to use the property, which matches the situation of (주)한공 paying the deposit.
Thus, the correct answer is (D).

Answer: (D)"""
    },
    {
        'role': 'user',
        'content':"""질문 : 암호 프로토콜 서비스에 대한 설명 중 옳지 않은 것은?
        선택지 : 
        A : 비밀성 : 자료 유출의 방지
        B : 접근제어 : 프로토콜 데이터 부분의 접근 제어
        C : 무결성 : 메시지의 변조를 방지
        D : 부인봉쇄 : 송수신 사실의 부정 방지
        """
    },
    {
        'role': 'assistant',
        'content': """Question: Which of the following statements about cryptographic protocol services is NOT correct?

        Options:
        A : Confidentiality: Prevention of data leakage
        B : Access control: Access control of protocol data parts
        C : Integrity: Prevention of message tampering
        D : Non-repudiation: Prevention of denial of sending or receiving

Let's think step by step. To understand the correct definition of each option, We refer to Wikipedia articles on computer science knowledge for help.

(A) Confidentiality: This means that data is protected from unauthorized access during transmission, so only the intended recipient can understand the content. This matches the description of "prevention of data leakage."

(B) Access Control: This refers to managing access to data or resources, typically by verifying user permissions and determining whether they can access specific data or resources. However, the description "access control of protocol data parts" is not accurate. Access control generally applies to data or resources broadly, not specifically to protocol data parts.

(C) Integrity: This ensures that data is protected from being altered or damaged during transmission, which matches the description of "prevention of message tampering."

(D) Non-repudiation: This ensures that the sender or receiver cannot deny having sent or received the data, which matches the description of "prevention of denial of sending or receiving."

Therefore, the incorrect description is (B) Access control: access control of protocol data parts. The description of "protocol data parts" does not align with the general concept of access control.

So, the correct answer is:

Answer: (B)"""
    },
    {
        'role': 'user',
        'content':"""신 교육 사회학에 대한 설명으로 옳지 않은 것은?
        선택지 : 
        A : 학교 교육과정 또는 교육내용에 주목한다.
        B : 불평등의 문제를 학교 교육 안에서 찾는다.
        C : 학교에서 가르치는 지식의 사회적 성격을 탐구한다.
        D : 구조기능주의에 기반하여 교육의 사회적 기능을 탐구한다.
        """
    },
    {
        'role': 'assistant',
        'content': """Question: Which of the following statements about the new sociology of education is incorrect?

        Options:
        A : It focuses on the school curriculum or educational content.
        B : It looks for issues of inequality within school education.
        C : It explores the social nature of the knowledge taught in schools.
        D : It explores the social function of education based on structural functionalism.

Let's think step by step. We refer to Wikipedia articles on pedagogy knowledge for help. First, the new sociology of education is a sociological study of school education that explores how school education is socially constructed and what impact it has. Based on this, let's look at the options. (A) Focusing on the school curriculum or educational content is one of the main features of the new sociology of education. The new sociology of education argues that the curriculum is socially constructed, and thus the knowledge and values that students acquire are influenced by it. (B) Finding the problem of inequality within school education is also an important feature of the new sociology of education. The new sociology of education argues that school education contributes to reproducing social inequality. (C) Exploring the social nature of the knowledge taught in schools is a key claim of the new sociology of education. The new sociology of education argues that knowledge is socially constructed and that the knowledge taught in schools is also formed according to the social context. (D) Exploring the social function of education based on structural functionalism is not a feature of the new sociology of education. The new sociology of education is critical of structural functionalism and interprets the social function of education from a different perspective. Therefore, the correct answer is (D).

Answer: (D)"""
    },   
    {
        'role': 'user',
        'content':""""2020년 현재 미국의 일반적인 도덕 기준에 따르면, 주인공(나/내가/내가 사용하는)은 이 두 시나리오 중 어느 시나리오에서 명백히 도덕적으로 잘못된 행동을 하는가? 시나리오 1 | 나는 어린 아들에게 공공장소에서 하는 한 자기 자신을 만지는 것이 괜찮다고 말했다. 시나리오 2 | 나는 이웃 집 앞에 있는 꽃을 뽑아서 쓰레기통에 던졌다."
        선택지 : 
        A : 잘못, 잘못
        B : 잘못, 잘못 아님
        C : 잘못 아님, 잘못
        D : 잘못 아님, 잘못 아님
        """
    },
    {
        'role': 'assistant',
        'content': """Question: According to the general moral standards in the United States as of 2020, in which of the two scenarios does the protagonist (I/myself) clearly engage in morally wrong behavior? scenario 1 | I tell my young son it's okay to touch yourself as long as it's in public. scenario 2 | I pulled flowers from in front of the house next door and threw them in the trash."
                  
        Options:
        A : Wrong, Wrong
        B : Wrong, Not wrong
        C : Not wrong, Wrong
        D : Not wrong, Not wrong
                  
Let's think step by step. According to Wikipedia for ethics and moral standards in America
scenario 1 is deemed unacceptable. Firstly, privacy is a fundamental cornerstone of ethical standards. Thus, encouraging a child to engage in inherently private behaviors in a public setting can jeopardize the child's understanding of personal boundaries, privacy, and public decency. Additionally, certain aspects of our bodies and behaviors are widely regarded as private out of respect for societal norms, individual comfort, and public decency. Secondly, encouraging behavior that flouts these norms, particularly in a child who is learning acceptable societal behavior, is ethically problematic. It can lead to potential judgement, shame, or discomfort for both the child and others. Thirdly, it's also critical to consider the educational role parents play. It's important not only to teach children about their bodies and their functions, but also where it is appropriate to engage in certain activities, and where it isn't. This includes teaching children about privacy, respect for others, and public behaviors versus personal behaviors, which this scenario neglects. Lastly, the scenario undermines the principle of respect for others present in public spaces. A core component of ethical action is considering the impact our behaviors have on others. Encouraging a young child to engage in private behaviors in public not only encroaches on the comfort and rights of others who share that space, but also sets a harmful precedent for the child's understanding of social interactions.
scenario 2 is deemed unacceptable. This is because it implies the act of removing another person's property without their permission, which infringes upon the rights of ownership that the neighbors have over their property, as encoded in our societal norms. These plants may have been cultivated with care and effort, and wantonly removing them shows a disregard for the value of other people's work and effort. Moreover, throwing the flowers in the trash is a waste of natural resources. Flowers often serve a purpose beyond mere decoration, such as attracting pollinators or improving air quality. Discarding them frivolously seems to show a lack of respect and concern for our environment. Additionally, this act can be viewed as a form of trespassing, if you physically entered your neighbor's property to commit the deed, which infrally with privacy rights and personal boundaries, as it involves violating another person's private space without their consent. Lastly, this scenario also demonstrates a lack of empathy. Empathy implies understanding and sharing the feelings of others. By discarding the flowers, the actor is disregarding the feelings and preferences of their neighbour, who likely enjoyed the presence of those flowers. These reasons, founded on the principles of respect for personal property, environmental propriety, privacy, and empathy, make this scenario ethically unacceptable according to human commonsense.

Answer: (A)"""
},
    {
        'role': 'user',
        'content':"""질문 : 점 C(2, -1)을 중심으로 하고 반지름의 길이가 √17인 원 위의 점 P(3, 3)에서의 접선과 점 Q(6, -2)에서의
      접선이 만나는 점을 R이라 할 때, 사각형 CQRP의 넓이는?
        선택지 : 
        A : √17
        B : 2√17
        C : '17'
        D : '34'
        """
    },
    {
        'role': 'assistant',
        'content': """Question: Given a circle with center C(2, -1) and a radius of √17, if the tangents from point P(3, 3) and point Q(6, -2) intersect at point R, what is the area of the quadrilateral CQRP?

        Options:
        A : √17
        B : 2√17
        C : 17
        D : 34

Let's think step by step. We refer to Wikipedia articles on mathematical knowledge for help. The problem is to find the area of the quadrilateral CQRP where C(2, -1) is the center of a circle with a radius of √17, and the tangents from points P(3, 3) and Q(6, -2) intersect at point R. By using the property that the line segment connecting the center of the circle to the tangent point is perpendicular to the tangent, we first find the slope of the line connecting P and C. The slope of PC is (3 - (-1)) / (3 - 2) = 4, so the slope of the tangent at P is -1/4. Similarly, the slope of QC is (-2 - (-1)) / (6 - 2) = -1/4, so the slope of the tangent at Q is 4. The equations of the tangents at P and Q are y - 3 = -1/4 * (x - 3) and y + 2 = 4 * (x - 6) respectively. Solving these equations simultaneously gives us the intersection point R at (8/3, 5/3). The distances between C and P, Q, R are calculated as CP = CQ = √17, and CR = √((8/3 - 2)^2 + (5/3 - (-1))^2) = √17. Thus, the quadrilateral CQRP is a square with an area of 17. Therefore, the answer is (C).

Answer: (C)"""
},

    {"role": "user", "content": question},
        ],
    }

def send_request(question):
    payload = create_payload(question)
    start_time = time.time()
    response = requests.post(url, headers=headers, json=payload)
    end_time = time.time()
    elapsed_time = end_time - start_time
    response_json = response.json()
    tokens_used = response_json['usage']['total_tokens']
    answer = response_json['choices'][0]['message']['content']
    return tokens_used, elapsed_time, answer

total_tokens = 0
total_time = 0
results = []
start_time = time.time()

with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    for i, questions in enumerate(result):
        futures = {executor.submit(send_request, question): question for question in questions}
        for future in concurrent.futures.as_completed(futures):
            question = futures[future]
            try:
                print(questions.index(question)+i*n)
                tokens, elapsed_time, answer = future.result()
                total_tokens += tokens
                total_time += elapsed_time
                results.append({"question":question, "tokens":tokens, "elapsed_time":elapsed_time, "answer":answer, "index_x":questions.index(question)+i*n})
                print(f"Question: {question}\nTokens Used: {tokens}\nElapsed Time: {elapsed_time}\nAnswer: {answer}\n")
            except Exception as exc:
                print(f"Question: {question}\nGenerated an exception: {exc}")

end_time = time.time()
overall_time = end_time - start_time
tokens_per_second = total_tokens / overall_time if overall_time > 0 else 0
print(f"Total Tokens: {total_tokens}\nOverall Time: {overall_time}\nTokens Per Second: {tokens_per_second}")

7
Question: 질문 : "영화 '웨스트 사이드 스토리'의 마지막에서 토니를 죽이는 사람은 누구입니까?"
                선택지 : 
                A : 리프
                B : 치노
                C : 베르나르도
                D : 그가 스스로 죽습니다
                
Tokens Used: 5068
Elapsed Time: 21.846598148345947
Answer: Question: "In the end of the movie 'West Side Story', who kills Tony?"

        Options:
        A : Riff
        B : Chino
        C : Bernardo
        D : He kills himself

Let's think step by step. We refer to Wikipedia articles on film knowledge for help. In the movie "West Side Story," the character Tony is killed by Chino at the end of the film. Chino, a member of the Sharks gang, mistakenly believes that Tony is responsible for the death of Bernardo, the leader of the Sharks. In retaliation, Chino shoots and kills Tony.

Answer: (B)

0
Question: 질문 : "샌델은 유전자 강화의 사용이 본질적으로 잘못되었다고 믿습니다. 그 이유는"
                선택지 : 
                A : 그 사용이 안전하지 않을 것이다.
                B : 그 사용이 불공평할 것이다.
                C : 위의 두 가지 모두

In [6]:
df_results = pd.DataFrame(data=results,columns=['question','tokens','elapsed_time','answer','index_x','answer_num'])

In [7]:
df_results.sort_values(by="index_x", inplace=True)
df_results.set_index(keys='index_x',inplace=True)
df_results.reset_index(drop=False,inplace=True)

In [8]:
num_ind = []
for j, i in enumerate(df_results["answer"]):
    matches = re.findall(r'Answer:\s*\(([A-D]+)\)', i)
    # matches = re.findall(r'\(*([가-라])\)*', i)
    if not matches:
        print(j)
        matches = ["C"]
    num = change_choices[matches[-1]]
    num_ind.append(num)

In [9]:
df_results["answer_num"] = num_ind

In [10]:
df_test = pd.read_csv("/workspace/test/sample_submission.csv")

In [ ]:
df_test["answer"] = num_ind

In [ ]:
df_results.to_csv("test_pred.csv")
# df_ans.to_csv("test_pred_only_num.csv")
df_test.to_csv("submission.csv",index=False)

프롬프트 출처
https://huggingface.co/datasets/HAERAE-HUB/KMMLU-HARD
https://huggingface.co/datasets/yc4142/ethics-CoT